# Priority Weighting: τ/ρ for Path Selection

This notebook demonstrates priority-weighted navigation using τ/ρ thresholds from **DOCS/PRIORITY_WEIGHTING.md**.

## Use Cases

1. **Language Generation**: Next-token selection
2. **Semantic Search**: Related concept discovery
3. **Recommendation Systems**: Content filtering
4. **Knowledge Navigation**: Path finding in concept graphs

In [12]:
import sys
sys.path.insert(0, '/home/alexmy/SGS/SGS_lib/hllset_manifold')

from core.hrt import (
    HRTConfig, HRT,
    BasicHLLSet
)
from core.kernel import Kernel
from core.hllset import HLLSet

kernel = Kernel()
print("✓ Framework loaded")

✓ Framework loaded


## Priority Function

**Formula:**

```text
Priority(u → v) = BSS_τ(u,v) - λ·BSS_ρ(u,v)
```

where:
- **BSS_τ(u,v) ∈ [0,1]**: Inclusion similarity (overlap)
- **BSS_ρ(u,v) ∈ [0,1]**: Exclusion similarity (divergence)
- **λ ∈ [0,1]**: Balance factor

**Interpretation:**
- High BSS_τ → High priority (strong connection)
- High BSS_ρ → Lower priority (penalize divergence)
- λ controls trade-off

In [13]:
# Configuration
config = HRTConfig(
    p_bits=8,
    h_bits=16,
    tau=0.7,      # Morphism requires 70% inclusion
    rho=0.3,      # And at most 30% exclusion
    epsilon=0.1
)

print(f"Configuration:")
print(f"  τ = {config.tau} (inclusion threshold)")
print(f"  ρ = {config.rho} (exclusion threshold)")
print(f"  Dimension: {config.dimension}")

Configuration:
  τ = 0.7 (inclusion threshold)
  ρ = 0.3 (exclusion threshold)
  Dimension: 4098


## Use Case 1: Language Generation

Given current context, select next token by priority.

In [3]:
# Create context (current sentence)
context_tokens = ['quantum', 'mechanics', 'describes', 'behavior']
context_hllset = kernel.absorb(context_tokens)
context = BasicHLLSet(index=0, is_row=True, hllset=context_hllset, config=config)

# Candidate next tokens
candidates_data = [
    (['of', 'particles', 'at', 'atomic'], 'Continuation'),
    (['superposition', 'entanglement', 'quantum'], 'Technical'),
    (['classical', 'deterministic', 'macroscopic'], 'Contrast'),
    (['wave', 'function', 'collapse', 'measurement'], 'Related'),
    (['pizza', 'restaurant', 'delivery', 'food'], 'Unrelated')
]

print(f"Context: {' '.join(context_tokens)}")
print(f"\nCandidate next tokens:\n")

Context: quantum mechanics describes behavior

Candidate next tokens:



In [4]:
# Compute priorities with different λ values
lambda_values = [0.3, 0.5, 0.7]

for lambda_weight in lambda_values:
    print(f"\n{'='*70}")
    print(f"λ = {lambda_weight} (balance factor)")
    print(f"{'='*70}\n")
    
    priorities = []
    for idx, (tokens, label) in enumerate(candidates_data, start=1):
        candidate_hllset = kernel.absorb(tokens)
        candidate = BasicHLLSet(index=idx, is_row=True, hllset=candidate_hllset, config=config)
        bss_t = context.bss_tau(candidate)
        bss_r = context.bss_rho(candidate)
        priority = bss_t - lambda_weight * bss_r
        passes_filter = context.has_morphism_to(candidate)
        priorities.append((label, bss_t, bss_r, priority, passes_filter))
    
    # Sort by priority
    priorities.sort(key=lambda x: x[3], reverse=True)
    
    print(f"{'Candidate':<15} {'BSS_τ':<8} {'BSS_ρ':<8} {'Priority':<10} {'Passes τ/ρ'}")
    print("-" * 70)
    
    for label, bss_t, bss_r, pri, passes in priorities:
        status = "✓" if passes else "✗"
        print(f"{label:<15} {bss_t:<8.3f} {bss_r:<8.3f} {pri:<10.3f} {status}")
    
    # Select top candidate that passes filter
    selected = next((label for label, _, _, _, passes in priorities if passes), None)
    if selected:
        print(f"\n→ Selected: {selected}")
    else:
        print(f"\n→ No candidate passes τ/ρ filter")


λ = 0.3 (balance factor)

Candidate       BSS_τ    BSS_ρ    Priority   Passes τ/ρ
----------------------------------------------------------------------
Continuation    0.000    1.000    -0.300     ✗
Related         0.000    1.000    -0.300     ✗
Unrelated       0.000    1.000    -0.300     ✗
Technical       0.000    1.334    -0.400     ✗
Contrast        0.000    1.334    -0.400     ✗

→ No candidate passes τ/ρ filter

λ = 0.5 (balance factor)

Candidate       BSS_τ    BSS_ρ    Priority   Passes τ/ρ
----------------------------------------------------------------------
Continuation    0.000    1.000    -0.500     ✗
Related         0.000    1.000    -0.500     ✗
Unrelated       0.000    1.000    -0.500     ✗
Technical       0.000    1.334    -0.667     ✗
Contrast        0.000    1.334    -0.667     ✗

→ No candidate passes τ/ρ filter

λ = 0.7 (balance factor)

Candidate       BSS_τ    BSS_ρ    Priority   Passes τ/ρ
----------------------------------------------------------------------


## Use Case 2: Semantic Search

Find related concepts in knowledge base.

In [5]:
# Query concept
query_tokens = ['machine', 'learning', 'neural', 'network']
query_hllset = kernel.absorb(query_tokens)
query = BasicHLLSet(index=0, is_row=True, hllset=query_hllset, config=config)

# Knowledge base
kb_data = [
    (['deep', 'learning', 'neural', 'network', 'training'], 'DeepLearning'),
    (['supervised', 'learning', 'classification', 'regression'], 'Supervised'),
    (['reinforcement', 'learning', 'agent', 'reward'], 'Reinforcement'),
    (['natural', 'language', 'processing', 'text'], 'NLP'),
    (['computer', 'vision', 'image', 'recognition'], 'Vision'),
    (['database', 'query', 'sql', 'relational'], 'Database')
]

print(f"Query: {' '.join(query_tokens)}")
print(f"\nSearching knowledge base...\n")

Query: machine learning neural network

Searching knowledge base...



In [6]:
# Rank by priority
lambda_weight = 0.5
results = []

for idx, (tokens, label) in enumerate(kb_data, start=1):
    doc_hllset = kernel.absorb(tokens)
    doc = BasicHLLSet(index=idx, is_row=True, hllset=doc_hllset, config=config)
    bss_t = query.bss_tau(doc)
    bss_r = query.bss_rho(doc)
    priority = bss_t - lambda_weight * bss_r
    passes = query.has_morphism_to(doc)
    results.append((label, tokens, bss_t, bss_r, priority, passes))

# Sort by priority
results.sort(key=lambda x: x[4], reverse=True)

print(f"Search Results (λ={lambda_weight}):\n")
print(f"{'Rank':<6} {'Document':<20} {'BSS_τ':<8} {'BSS_ρ':<8} {'Priority':<10} {'Passes'}")
print("-" * 80)

for rank, (label, tokens, bss_t, bss_r, pri, passes) in enumerate(results, 1):
    status = "✓" if passes else "✗"
    print(f"{rank:<6} {label:<20} {bss_t:<8.3f} {bss_r:<8.3f} {pri:<10.3f} {status}")
    if rank <= 3:  # Show top 3 tokens
        print(f"       {tokens}")

Search Results (λ=0.5):

Rank   Document             BSS_τ    BSS_ρ    Priority   Passes
--------------------------------------------------------------------------------
1      DeepLearning         0.000    0.800    -0.400     ✗
       ['deep', 'learning', 'neural', 'network', 'training']
2      Supervised           0.000    1.000    -0.500     ✗
       ['supervised', 'learning', 'classification', 'regression']
3      Reinforcement        0.000    1.000    -0.500     ✗
       ['reinforcement', 'learning', 'agent', 'reward']
4      NLP                  0.000    1.000    -0.500     ✗
5      Vision               0.000    1.000    -0.500     ✗
6      Database             0.000    1.000    -0.500     ✗


## Use Case 3: Path Finding

Navigate through concept graph using priority weights.

In [7]:
# Create concept graph
concepts = [
    (['quantum', 'mechanics', 'physics'], 'QuantumMechanics'),
    (['wave', 'function', 'superposition'], 'WaveFunction'),
    (['measurement', 'collapse', 'observation'], 'Measurement'),
    (['entanglement', 'nonlocality', 'quantum'], 'Entanglement'),
    (['decoherence', 'environment', 'interaction'], 'Decoherence'),
    (['classical', 'limit', 'correspondence'], 'ClassicalLimit')
]

nodes = [BasicHLLSet(index=idx, is_row=True, hllset=kernel.absorb(tokens), config=config)
         for idx, (tokens, label) in enumerate(concepts)]

print(f"Concept graph with {len(nodes)} nodes:\n")
for i, (tokens, label) in enumerate(concepts):
    print(f"  {i}: {label} - {tokens}")

Concept graph with 6 nodes:

  0: QuantumMechanics - ['quantum', 'mechanics', 'physics']
  1: WaveFunction - ['wave', 'function', 'superposition']
  2: Measurement - ['measurement', 'collapse', 'observation']
  3: Entanglement - ['entanglement', 'nonlocality', 'quantum']
  4: Decoherence - ['decoherence', 'environment', 'interaction']
  5: ClassicalLimit - ['classical', 'limit', 'correspondence']


In [8]:
# Build adjacency based on priority
lambda_weight = 0.5
min_priority = 0.3  # Threshold for edge existence

print(f"\nBuilding adjacency (λ={lambda_weight}, min_priority={min_priority}):\n")

edges = []
for i, node_i in enumerate(nodes):
    for j, node_j in enumerate(nodes):
        if i < j:  # Undirected graph
            bss_t = node_i.bss_tau(node_j)
            bss_r = node_i.bss_rho(node_j)
            priority = bss_t - lambda_weight * bss_r
            passes = node_i.has_morphism_to(node_j)
            
            if priority >= min_priority and passes:
                edges.append((i, j, priority, concepts[i][1], concepts[j][1]))

# Sort by priority
edges.sort(key=lambda x: x[2], reverse=True)

print(f"Found {len(edges)} edges:\n")
for i, j, pri, label_i, label_j in edges:
    print(f"  {label_i:<20} ↔ {label_j:<20} (priority={pri:.3f})")


Building adjacency (λ=0.5, min_priority=0.3):

Found 0 edges:



In [9]:
# Greedy path finding with priority
def find_path_greedy(start_idx, goal_idx, nodes, edges_list, max_steps=10):
    """Find path using greedy priority-weighted selection."""
    current = start_idx
    path = [current]
    visited = {current}
    
    for _ in range(max_steps):
        if current == goal_idx:
            return path, True
        
        # Find neighbors with priorities
        neighbors = []
        for i, j, pri, _, _ in edges_list:
            if i == current and j not in visited:
                neighbors.append((j, pri))
            elif j == current and i not in visited:
                neighbors.append((i, pri))
        
        if not neighbors:
            return path, False  # Dead end
        
        # Select highest priority
        next_node, next_pri = max(neighbors, key=lambda x: x[1])
        path.append(next_node)
        visited.add(next_node)
        current = next_node
    
    return path, False  # Max steps reached

# Find path from QuantumMechanics to ClassicalLimit
start_idx = 0  # QuantumMechanics
goal_idx = 5   # ClassicalLimit

path, success = find_path_greedy(start_idx, goal_idx, nodes, edges, max_steps=10)

print(f"\nPath from {concepts[start_idx][1]} to {concepts[goal_idx][1]}:\n")
if success:
    print(f"✓ Found path with {len(path)} steps:\n")
    for step, idx in enumerate(path):
        print(f"  {step}: {concepts[idx][1]}")
        if step < len(path) - 1:
            # Find edge weight
            next_idx = path[step + 1]
            edge_pri = next((pri for i, j, pri, _, _ in edges 
                           if (i == idx and j == next_idx) or (j == idx and i == next_idx)), 
                          None)
            if edge_pri:
                print(f"      ↓ (priority={edge_pri:.3f})")
else:
    print(f"✗ No path found (reached {concepts[path[-1]][1]})")


Path from QuantumMechanics to ClassicalLimit:

✗ No path found (reached QuantumMechanics)


## Selection Strategies

**1. Greedy (deterministic):**
```python
next_node = argmax_v Priority(u → v)
```

**2. Stochastic (probabilistic):**
```python
P(v | u) = softmax(Priority(u → v) / temperature)
```

**3. Beam Search (keep top-k):**
```python
candidates = top_k([Priority(u → v) for v in neighbors])
```

In [10]:
import torch
import torch.nn.functional as F

def stochastic_select(current, candidates, lambda_weight=0.5, temperature=1.0):
    """Stochastic selection using softmax over priorities."""
    priorities = []
    for cand in candidates:
        bss_t = current.bss_tau(cand)
        bss_r = current.bss_rho(cand)
        pri = bss_t - lambda_weight * bss_r
        priorities.append(pri)
    
    # Softmax with temperature
    logits = torch.tensor(priorities) / temperature
    probs = F.softmax(logits, dim=0)
    
    return candidates, priorities, probs.tolist()

# Demonstrate stochastic selection
current = nodes[0]  # QuantumMechanics
candidates_idx = [1, 2, 3]  # WaveFunction, Measurement, Entanglement
candidates_nodes = [nodes[i] for i in candidates_idx]

print(f"Stochastic selection from {concepts[0][1]}:\n")

for temp in [0.5, 1.0, 2.0]:
    cands, pris, probs = stochastic_select(current, candidates_nodes, 
                                          lambda_weight=0.5, temperature=temp)
    
    print(f"Temperature = {temp}:")
    for i, (idx, pri, prob) in enumerate(zip(candidates_idx, pris, probs)):
        print(f"  {concepts[idx][1]:<20} priority={pri:.3f}, P={prob:.3f}")
    print()

Stochastic selection from QuantumMechanics:

Temperature = 0.5:
  WaveFunction         priority=-0.500, P=0.333
  Measurement          priority=-0.500, P=0.333
  Entanglement         priority=-0.500, P=0.333

Temperature = 1.0:
  WaveFunction         priority=-0.500, P=0.333
  Measurement          priority=-0.500, P=0.333
  Entanglement         priority=-0.500, P=0.333

Temperature = 2.0:
  WaveFunction         priority=-0.500, P=0.333
  Measurement          priority=-0.500, P=0.333
  Entanglement         priority=-0.500, P=0.333



## Summary

**Priority Weighting with τ/ρ:**

1. **Formula:** Priority = BSS_τ - λ·BSS_ρ
   - Balances inclusion (BSS_τ) vs exclusion (BSS_ρ)
   - λ controls trade-off

2. **Filtering:** Candidates must pass τ/ρ thresholds
   - BSS_τ ≥ τ (sufficient overlap)
   - BSS_ρ ≤ ρ (limited divergence)

3. **Strategies:**
   - **Greedy:** Select highest priority (deterministic)
   - **Stochastic:** Sample from softmax distribution
   - **Beam:** Keep top-k candidates

4. **Applications:**
   - Language generation (next token)
   - Semantic search (document ranking)
   - Knowledge navigation (path finding)
   - Recommendation systems (item filtering)

**Key Insight:** τ/ρ are the **mechanics** of contextual selection, enabling priority-weighted navigation through semantic space.

**References:**
- **DOCS/PRIORITY_WEIGHTING.md**: Complete API reference
- **DOCS/HRT_LATTICE_THEORY.md**: Theoretical foundations
- **DOCS/CONTEXTUAL_SELECTION_MECHANICS.md**: Principle → Mechanics → Result